In [1]:
import random
import os
import json
import pyarrow as pa
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
train_data = list(
    map(json.loads, open(f"/root/thesis/dataset/cosmos_anns_acm/acm_anns/train_data.json").readlines())
)
# test_data = list(
#     map(json.loads, open(f"/root/thesis/dataset/cosmos_anns_acm/acm_anns/train_data.json").readlines())
# )
val_data = list(map(json.loads, open(f"/root/thesis/dataset/cosmos_anns_acm/acm_anns/val_data.json").readlines()))

In [4]:
def gen(the_data):
    the_data = np.random.choice(the_data, size=int(len(the_data)*50/100))

    l=[]
    for i, data in enumerate(tqdm(the_data)):
        if len(data['articles']) > 1:
            caps = [article['caption_modified'] for article in data['articles']]
            caps_pairs = [[data['img_local_path'],[a], [b],[1]] for idx, a in enumerate(caps) for b in caps[idx + 1:] if a!=b]
            l += caps_pairs

        # wrong case
        for z in [0,1]:
            idx = random.sample(range(0,len(data['articles'])),1)[0]
            cap1 = data['articles'][idx]['caption_modified']

            r_id = random.sample(range(0,len(train_data)),1)[0]
            if r_id==i:
                continue

            data2 =  train_data[r_id]
            idx2 = random.sample(range(0,len(data2['articles'])),1)[0]
            cap2 = data2['articles'][idx2]['caption_modified']
            caps_pairs = [[data['img_local_path'],[cap1],[cap2],[0]]]
            l+=caps_pairs
    return l

In [6]:
np.random.seed(42)

dataframe_train = pd.DataFrame(
    gen(train_data), columns=["image", "caption_1", "caption_2", "label"],
)

dataframe_val = pd.DataFrame(
    gen(val_data), columns=["image", "caption_1", "caption_2", "label"],
)

100%|██████████| 20503/20503 [00:00<00:00, 25232.92it/s]


In [7]:
def load_image(path):
   try:
      with open(path, "rb") as fp:
        return fp.read()
   except:
      return None

In [8]:
tqdm.pandas()
dataframe_train['image'] = dataframe_train['image'].progress_apply(lambda x: load_image(
    os.path.join('/root/thesis/dataset', x)
    ))
dataframe_val['image'] = dataframe_val['image'].progress_apply(lambda x: load_image(
    os.path.join('/root/thesis/dataset', x)
    ))

/root/anaconda3/envs/vilt2/lib/python3.7/site-packages/tqdm/std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 73979/73979 [00:10<00:00, 7382.46it/s]


In [9]:
split = 'train'

table = pa.Table.from_pandas(dataframe_train)
with pa.OSFile(f"dataset/iou/cosmos_{split}.arrow", "wb") as sink:
    with pa.RecordBatchFileWriter(sink, table.schema) as writer:
        writer.write_table(table)

split = 'val'

table = pa.Table.from_pandas(dataframe_val)
with pa.OSFile(f"dataset/iou/cosmos_{split}.arrow", "wb") as sink:
    with pa.RecordBatchFileWriter(sink, table.schema) as writer:
        writer.write_table(table)